In [2]:
!python -m pip install pyserial


     |████████████████████████████████| 194kB 2.7MB/s 


In [3]:


#!/usr/bin python3
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 12 19:00:49 2019

@author: neo
"""
import random
import sys
import os
import time
import serial
import socket
import math
from tkinter import *
from tkinter import ttk
from tkinter.ttk import Combobox
from tkinter import filedialog
import numpy as np
import matplotlib

#import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure

class MainWindow:
    def __init__(self, master):
        self.master = master
        self.ttle=master.title("AUTOPATTERN")
        master.resizable(width=FALSE, height=FALSE)
        master.geometry("650x450")
        self.modelabel = Label(master, text="Mode: ")
        self.modelabel.grid(row=0,column=0,pady=3, padx=5, sticky = W)
        self.modecombo=Combobox(master)
        self.modecombo['values']=("XZ Plane Co","XZ Plane Cx","YZ Plane Co","YZ Plane Cx","XY Plane Co","XY Plane Cx")
        self.modecombo.current(0)
        self.modecombo.grid(row=0,column=1,sticky=W)
        self.steplabel=Label(master,height=3, text="Step angle (degree): ")
        self.steplabel.grid(row=1,column=0,pady=2,padx=5,sticky = W)
        self.angleentry=Entry(master,width=5)
        self.angleentry.insert(END,"5")
        self.angleentry.grid(row=1,column=1,sticky=W)
        self.direction = BooleanVar()
        self.direction.set(0)
        self.cwradiobutton=Radiobutton(master, text='Clockwise', variable=self.direction, value=0)
        self.cwradiobutton.grid(row=2, column=0, padx=5,sticky=W)
        self.ccwradiobutton=Radiobutton(master, text='Counterclockwise', variable=self.direction, value=1)
        self.ccwradiobutton.grid(row=2, column=1, sticky=W)
        self.rotatebutton=Button(master,height=5,text="Rotate",command=self.rotate)
        self.rotatebutton.grid(row=3,column=0,columnspan=2,rowspan=5,padx=5,sticky=EW)
        self.patternbutton = Button(master,height=12, text="Pattern", command=self.pattern)
        self.patternbutton.grid(row=8,column=0,columnspan=2,rowspan=5,padx=5,sticky=EW)
        
        self.theta=np.arange(0,360,int(self.angleentry.get()))
        self.r=np.sin(self.theta)*np.cos(self.theta)
        
        self.fig=Figure(figsize=(4.5,5))
        self.fig.suptitle("Radiation pattern preview")
        self.ax=self.fig.add_axes([0.1,0.1,0.8,0.8],polar=True)
        self.plt=self.ax.plot(self.theta,self.r)
        self.canvas=FigureCanvasTkAgg(self.fig,master=root)
        self.canvas.get_tk_widget().grid(row=0,column=2,rowspan=12,padx=2.5,sticky=EW)
        self.ax.clear()
        self.canvas.draw()
        
        self.scrollvar=IntVar()
        self.scrollvar.set(0)
        self.previewscroll=Scale(master,variable=self.scrollvar,from_ = 1, to = 201, orient=HORIZONTAL)
        self.previewscroll.grid(row=12,column=2,padx=2.5,sticky=EW)
        self.scrollvar.trace("w",self.previewfreqchanged)
        
    def rotate(self):        
        self.modecombo.configure(state=DISABLED)
        self.angleentry.configure(state=DISABLED)
        self.cwradiobutton.configure(state=DISABLED)
        self.ccwradiobutton.configure(state=DISABLED)
        self.rotatebutton.configure(state=DISABLED)
        self.patternbutton.configure(state=DISABLED)
        motor = serial.Serial(port='/dev/ttyUSB0',	baudrate=4800, parity=serial.PARITY_ODD, stopbits=serial.STOPBITS_ONE, bytesize=serial.SEVENBITS, timeout=5)
        motor.close()
        motor.open()
        motor.flushInput()
        time.sleep(.3)
        motor.flushOutput()
        self.angle=self.angleentry.get()
        os.system('clear')
        os.system('clear')
        if self.direction.get(): #counter clockwise rotation
            motor.write(str.encode('D'+self.angle.zfill(3)+'\r'))
            time.sleep(.25*float(self.angle))
            motor.close()
        else: #clockwide rotation
            motor.write(str.encode('D'+self.angle.zfill(3)+'-'+'\r'))
            time.sleep(.25*float(self.angle))
            motor.close()            
        self.modecombo.configure(state=NORMAL)
        self.angleentry.configure(state=NORMAL)
        self.cwradiobutton.configure(state=NORMAL)
        self.ccwradiobutton.configure(state=NORMAL)
        self.rotatebutton.configure(state=NORMAL)
        self.patternbutton.configure(state=NORMAL)
        
    def pattern(self):
        foldername=filedialog.askdirectory()
        if len(foldername)==0:
            print("No file selected!")
            return
        print(foldername+ " selected as measurement folder")
        print("Measurement started. Please wait...")
        motor = serial.Serial(port='/dev/ttyUSB0',	baudrate=4800, parity=serial.PARITY_ODD, stopbits=serial.STOPBITS_ONE, bytesize=serial.SEVENBITS, timeout=5)
        motor.close()
        motor.open()
        motor.flushInput()
        time.sleep(.3)
        motor.flushOutput()
        analyzer=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        analyzer.connect(("192.168.1.1",5025))
        analyzer.settimeout(1)
        analyzer.send(str.encode("INIT:CONT OFF\n"))	
        time.sleep(.3)
        analyzer.send(str.encode("SENS:FREQ:STAR?\n"))	
        time.sleep(.3)
        f1=analyzer.recv(201*1024)
        print("Start= "+str(float(f1)))
        analyzer.send(str.encode("SENS:FREQ:STOP?\n"))	
        time.sleep(.3)
        f2=analyzer.recv(201*1024)
        print("Stop= "+str(float(f2)))
        analyzer.send(str.encode("SENS:SWE:POIN?\n"))	
        time.sleep(.3)
        fp=analyzer.recv(201*1024)
        print("Points= "+str(float(fp)))
        f=np.linspace(float(f1),float(f2),num=int(fp))
        analyzer.send(str.encode("INIT:CONT ON\n"))
        time.sleep(.3)
        analyzer.send(str.encode("CALC:PAR:CAT?\n"))	
        time.sleep(1)
        x=analyzer.recv(10000*1024)
        analyzer.send(str.encode('CALC:PAR:SEL '+str(x[1:-1-5])[1:]+'\n'))
        time.sleep(.3)
        x=f/1e9
        a = len(x)
        self.previewscroll.configure(from_ = 1, to = a)
        Max=360
        angle=self.angleentry.get()
        step=float(angle)
        count=0
        self.modecombo.configure(state=DISABLED)
        self.angleentry.configure(state=DISABLED)
        self.cwradiobutton.configure(state=DISABLED)
        self.ccwradiobutton.configure(state=DISABLED)
        self.rotatebutton.configure(state=DISABLED)
        while count<=Max-step:
            print("Current angle= "+str(count))
            analyzer.send(str.encode("INIT:CONT OFF\n"))
            time.sleep(.1)
            analyzer.send(str.encode("INIT:IMM;*wai\n"))
            time.sleep(.1)		
            analyzer.send(str.encode("CALC:DATA? FDATA;*OPC?\n"))
            data=str()
            while 1:
                data+=str(analyzer.recv(1000))[2:-1]
                if data.find(';+1')>0:
                    break;
            data=data.replace(';+1','')[0:-2]
            fdata=np.array("".join(data).split(',')).astype(np.float)
            x=np.append(x,fdata,axis=0)
            analyzer.send(str.encode("INIT:CONT ON\n"))
            if self.direction.get(): #counter clockwise rotation
                motor.write(str.encode('D'+angle.zfill(3)+'\r'))
                time.sleep(.25*step)
            else: #clockwide rotation
                motor.write(str.encode('D'+angle.zfill(3)+'-'+'\r'))
                time.sleep(.25*step)
            time.sleep(.3*step)
            motor.flushInput()
            motor.flushOutput()
            count=count+step
        x=np.reshape(x,(int(Max/step)+1,a)) 
        theta=np.linspace(0,Max-int(step),num=int(Max/step))
        theta=np.append(0,theta)
        theta=np.reshape(theta,(len(theta),1))
        x=np.append(theta,x,1)
        np.savetxt(foldername+'/'+self.modecombo.get()+'.csv',x,fmt='%f',delimiter=',')
        print('File written successfully !')
        C=str(int(count))    	
        if self.direction.get(): #clockwise return
            print("Returning clockwise...")
            motor.write(str.encode('D'+C.zfill(3)+'-'+'\r'))
            status=str()
            while 1:
                status=str(motor.readline())[2:-1]
                if status:
                    print("Returned to initial position")
                    break
        else: #counter clockwide return
            print("Returning counter clockwise...")
            motor.write(str.encode('D'+C.zfill(3)+'\r'))
            status=str()
            while 1:
                status=str(motor.readline())[2:-1]
                if status:
                    print("Returned to initial position")
                    break
        print('Radiation Pattern Completed !')
        motor.close()
        self.modecombo.configure(state=NORMAL)
        self.angleentry.configure(state=NORMAL)
        self.cwradiobutton.configure(state=NORMAL)
        self.ccwradiobutton.configure(state=NORMAL)
        self.rotatebutton.configure(state=NORMAL)
        
    def exitbuttonpressed(self):
        print("Paulbert")
        self.master.destroy()        

    def previewfreqchanged(self,*args):
        self.fig.suptitle("Radiation Pattern at "+str(self.previewscroll.get())+" GHz")
        self.theta=np.arange(0,360,float(self.angleentry.get()))
        self.r=np.sin(self.theta)*np.cos(self.theta)
        self.ax.clear()
        self.plt=self.ax.plot(self.theta,self.r**self.previewscroll.get())
        self.canvas.draw()
        
root = Tk()
my_gui = MainWindow(root)
root.mainloop()


ImportError: ignored